pip install gymnasium[box2d]
pip install stable-baselines3[extra]: The deep reinforcement learning library.
pip install huggingface_sb3:

Usando env.step(action), realizamos esta ação no ambiente e obtemos

- observation: O novo estado (st+1)
- reward:A recompensa que obtemos após executar a ação
- terminated: Indica se o episódio foi encerrado (o agente atingiu o estado terminal)
- truncated:Introduzido com esta nova versão, ele indica um limite de tempo ou se um agente sai dos limites do ambiente, por exemplo.
- info: Um dicionário que fornece informações adicionais (depende do ambiente).

In [2]:
from tensorboard import notebook
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import StopTrainingOnNoModelImprovement, StopTrainingOnRewardThreshold, EvalCallback, CheckpointCallback, CallbackList
import gymnasium as gym
import os
from datetime import datetime

In [3]:
env = gym.make('LunarLander-v3',  render_mode="rgb_array")

In [4]:


# Obter o timestamp atual com milissegundos
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S-%f")

# Criar diretórios para armazenar modelos e logs
model_dir = f"models"  # Diretório onde os melhores modelos serão salvos.
log_dir = "logs"  # Diretório onde os logs de treinamento serão salvos.
os.makedirs(model_dir, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)

# Callback para salvar checkpoints durante o treinamento
checkpoint_callback = CheckpointCallback(
    save_freq=100,  # Salvar o modelo a cada 100 passos de treinamento. 
    # Exemplo: o modelo será salvo como "checkpoint/rl_model_step_100.zip".
    save_path="./checkpoint/",  # Caminho onde os arquivos de checkpoint serão salvos.
    # Exemplo: Arquivos salvos no diretório "checkpoint".
    name_prefix="rl_model",  # Prefixo dos arquivos de modelo. 
    # Exemplo: Arquivos salvos como "rl_model_step_x.zip".
    save_replay_buffer=True,  # Salvar o buffer de replay (usado em algoritmos off-policy como DQN/SAC). 
    # Exemplo: Um arquivo adicional será salvo com o buffer de replay.
    save_vecnormalize=True  # Salvar o estado da normalização do vetor (se VecNormalize for usado). 
    # Exemplo: Inclui os parâmetros de normalização ao salvar o modelo.
)

# Callback para interromper o treinamento ao atingir uma recompensa mínima
callback_on_best = StopTrainingOnRewardThreshold(
    reward_threshold=300,  # Interromper o treinamento se o modelo atingir recompensa média de pelo menos 300. 
    # Exemplo: Se a avaliação produzir uma recompensa de 310, o treinamento será interrompido.
    verbose=1  # Mostrar mensagens no console quando a condição for atingida. 
    # Exemplo: "New best model found with reward: 310. Training will stop."
)

# Callback para parar o treinamento se não houver melhorias
stop_train_callback = StopTrainingOnNoModelImprovement(
    max_no_improvement_evals=5,  # Número máximo de avaliações sem melhorias antes de interromper o treinamento. 
    # Exemplo: Se o modelo não melhorar em 5 avaliações consecutivas, o treinamento será interrompido.
    min_evals=10000,  # Avaliações mínimas antes de verificar a condição de interrupção. 
    # Exemplo: O critério de melhoria só será avaliado após 10.000 passos de treinamento.
    verbose=1  # Mostrar mensagens no console quando a condição for atingida. 
    # Exemplo: "No improvement in the last 5 evaluations. Stopping training."
)

# Callback para avaliação do modelo durante o treinamento
eval_callback = EvalCallback(
    env,  # Ambiente usado para avaliação. 
    # Exemplo: Se o ambiente for `CartPole-v1`, o modelo será avaliado neste ambiente.
    eval_freq=10000,  # Frequência de avaliação, em passos de treinamento. 
    # Exemplo: O modelo será avaliado a cada 10.000 passos.
    callback_on_new_best=callback_on_best,  # Callback para executar ao encontrar um novo melhor modelo. 
    # Exemplo: Se o modelo atingir uma nova melhor recompensa, `StopTrainingOnRewardThreshold` será acionado.
    callback_after_eval=stop_train_callback,  # Callback para executar após cada avaliação. 
    # Exemplo: Após cada avaliação, verifica se o modelo parou de melhorar.
    verbose=1,  # Mostrar mensagens no console sobre o progresso da avaliação. 
    # Exemplo: "Evaluation #1 - Reward: 250".
    best_model_save_path=os.path.join(model_dir, f"model_{current_time}"),  
    # Caminho para salvar o melhor modelo. 
    # Exemplo: Um modelo com desempenho superior será salvo em "PPO_models_current_time/model_2025-01-11_15-30-45-123456".
)

# Criar uma lista de callbacks para facilitar o gerenciamento
callback = CallbackList([checkpoint_callback, eval_callback])


In [6]:


env = gym.make('LunarLander-v3',  render_mode="rgb_array")
# Save a checkpoint every 1000 steps
env = gym.wrappers.RecordVideo(env=env, video_folder="./video_trainig/", name_prefix="test-video", episode_trigger=lambda x: x % 100 == 0)
# Instantiate the agent
env = Monitor(env)

model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_dir)


# Train the agent
model.learn(total_timesteps=int(1e10), tb_log_name=f"{model_dir}", callback=eval_callback)


Using cuda device
Wrapping the env in a DummyVecEnv.
Logging to logs\models_4


c:\Users\igor8\AppData\Local\Programs\Python\Python311\Lib\site-packages\gymnasium\wrappers\rendering.py:283: UserWarning: WARN: Overwriting existing videos at c:\Users\igor8\Desktop\project_reinforcemen_learning\RL-LunarLander-v2\teste_1_problem_LunarLander-v3\video_trainig folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


KeyboardInterrupt: 

In [7]:
# model.save("ppo_cartpole")

## Análise dos Gráficos de Treinamento

### 1. `rollout/ep_len_mean` (Comprimento Médio do Episódio)
#### **Descrição**
- Este gráfico mostra o comprimento médio dos episódios ao longo do treinamento.
- Um comprimento maior geralmente indica que o agente está melhorando seu desempenho no ambiente.

#### **Indicadores**
- **Bons:**
  - Aumento constante no comprimento médio ao longo do tempo.
  - Indica que o agente está sobrevivendo mais tempo ou completando tarefas de forma eficiente.
- **Ruins:**
  - Estagnação ou queda no comprimento médio.
  - Pode indicar que o agente está tendo dificuldades para aprender.

---

### 2. `rollout/ep_rew_mean` (Recompensa Média por Episódio)
#### **Descrição**
- Este gráfico mede a recompensa média obtida pelo agente em cada episódio.
- Recompensas mais altas indicam que o agente está aprendendo a maximizar o objetivo definido.

#### **Indicadores**
- **Bons:**
  - Recompensas crescentes ao longo do treinamento.
  - Indica que o agente está melhorando suas decisões no ambiente.
- **Ruins:**
  - Recompensas negativas ou que não apresentam crescimento.
  - Sinal de que o agente está estagnado ou não está aprendendo de forma eficaz.

---

### 3. `time/fps` (Frames por Segundo)
#### **Descrição**
- Este gráfico exibe a eficiência computacional do treinamento, medindo o número de frames por segundo (FPS) processados.

#### **Indicadores**
- **Bons:**
  - FPS elevado e estável (acima de 300).
  - Indica que o treinamento está sendo executado de forma eficiente.
- **Ruins:**
  - Quedas abruptas ou FPS consistentemente abaixo de 200.
  - Pode ser um indicativo de sobrecarga no hardware ou aumento da complexidade computacional.

---

### Conclusão Geral
1. **`rollout/ep_len_mean`**: O comprimento médio dos episódios está aumentando, o que é um sinal positivo de aprendizado.
2. **`rollout/ep_rew_mean`**: A recompensa média também está crescendo, indicando que o agente está melhorando seu desempenho.
3. **`time/fps`**: Apesar de uma queda gradual no FPS, ele permanece em um nível aceitável (~285 FPS), mantendo o treinamento eficiente.

Esses gráficos mostram que o treinamento está funcionando bem, tanto em termos de aprendizado do agente quanto de eficiência computacional.


In [8]:
##tensorboard --logdir=./teste_1_problem_LunarLander-v3/logs/
%load_ext tensorboard
%tensorboard --logdir ./logs/

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 25708), started 21:29:48 ago. (Use '!kill 25708' to kill it.)